In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import yfinance as yf
from scipy.optimize import minimize
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models, expected_returns

import PySimpleGUI as sg

%matplotlib inline
plt.style.use("seaborn-whitegrid")
plt.rcParams['figure.figsize'], plt.rcParams['figure.dpi'] = [15, 9], 120
pd.set_option('display.max_columns', None)  
np.set_printoptions(threshold=None) 

In [ ]:
def make_input_window(currencies):
    sg.theme('DarkTeal9')
    sg.set_options(font=('Helvetica', 12), element_padding=(10, 10), button_color=('#FFFFFF', '#333333'))
    layout = [
    [sg.Text('Select currencies:'), sg.Listbox(currencies, size=(20, len(currencies)), key='currencies', select_mode=sg.LISTBOX_SELECT_MODE_EXTENDED)],
    [
    [sg.Text('Input portfolio size: $'), sg.Input(k='portfolio_size', size=(20,1))]
    ],
    [
    [sg.Text('Input portfolio expected profit: %'), sg.Input(k='expected_profit', size=(20,1))]
    ],
    [
    [sg.Text('Add more currencies: (AUD, CNY, DKK, JPY)'), sg.Input(k='new_currencies', size=(20,1))]
    ],
    [
    [sg.Radio('Use portfolio risk minimization method', "RadioDemo", default=True, k='if_min_method')],
    [sg.Radio('Use Markowitz method', "RadioDemo", default=False, k='if_markowitz_method')]
    ],
    [sg.Checkbox('Select All', key='select_all', default=False)],
    [sg.Button('OK'), sg.Button('EXIT')]
    ]

    
    return sg.Window('Modelling currency basket', layout)


# risk basket func
def portfolio_volatility(weights, cov_matrix):
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

# limitation
def check_sum(weights):
    return np.sum(weights) - 1

# minimization of risk
def minimize_risk(weights, cov_matrix):
    return portfolio_volatility(weights, cov_matrix)



currencies = ['GBP', 'EUR', 'CHF', 'CAD', 'SEK']
window = make_input_window(currencies)

while True:
    event, values = window.read()

    if event == sg.WINDOW_CLOSED or event == 'EXIT':
        break
    
    if event == 'select_all':
        if values['select_all']:
            window['currencies'].update(set_to_index=list(range(len(currencies))))
        else:
            window['currencies'].update(set_to_index=[])
    
    if event == 'OK':
        if values['select_all']:
            selected_currencies = currencies
        else:
            selected_currencies = values['currencies']
        if values['new_currencies']:
            selected_currencies.extend(values['new_currencies'].split(' '))
        if values['portfolio_size']:
            portfolio_size = float(values['portfolio_size'])
        else:
            portfolio_size = 100_000
        if values['expected_profit']:
            expected_profit = float(values['expected_profit'])/100
        else:
            expected_profit = 0.02
        
        start_date = '2022-01-01'
        end_date = '2022-12-31'
        basket = pd.DataFrame()
        for currency in selected_currencies:
            rates = yf.download(f'{currency}=X', start=start_date, end=end_date)
            basket[currency] = rates['Close']


        if values['if_min_method']:
            print('\nPortfolio risk minimization method:\n')
            returns = np.log(basket / basket.shift(1)).dropna()
            cov_matrix = returns.cov()

            num_assets = len(basket.columns)
            bounds = tuple((0, 1) for x in range(num_assets))
            constraints = ({'type': 'eq', 'fun': check_sum})
            initial_guess = num_assets * [1 / num_assets]
            opt_results = minimize(minimize_risk, initial_guess, args=cov_matrix, method='SLSQP', bounds=bounds, constraints=constraints)
            weights = opt_results.x

            allocation = pd.DataFrame(weights, index=basket.columns, columns=['Allocation'])

            print('\nCurrency basket:\n')
            for i in range(len(allocation['Allocation'])):
                print(f'{allocation.index[i]}: {round(allocation["Allocation"][i]*100, 2)} %')

            position_size = {}
            for i in allocation.index:
                position_size[i] = allocation.loc[i]['Allocation'] * portfolio_size

            print("\nPortfolio:\n")
            for i in allocation.index:
                print(f"{i}: {round(position_size[i], 2)}$ ({round(allocation.loc[i]['Allocation']*100, 2)}%)")
            print(f"\nSum of investments: {int(round(sum(position_size.values()),2))}$")

            plt.bar(allocation.index, round(allocation['Allocation']*100, 2))
            plt.xlabel('Currency')
            plt.ylabel('Percentage')
            plt.title('Percentage currency basket')
            plt.show()
        else:
            print('\nMarkowitz method:\n')
            mu = expected_returns.mean_historical_return(basket)
            S = risk_models.sample_cov(basket)
            
            ef = EfficientFrontier(mu, S)
            if expected_profit > mu.max():
                print(f'Inputed expected portfolio profit ({expected_profit}) is bigger, than maximum possible profit ({mu.max()}), hence using almost max avaliable profit)\n')
                expected_profit = mu.max() - 0.001
            ef.efficient_return(target_return=expected_profit)
            cleaned_weights = ef.clean_weights()
            ef.portfolio_performance(verbose=True)
            print('\nCurrency basket:\n')
            for currency, weight in cleaned_weights.items():
                print(f"{currency}: {weight*100:.2f}%")
            
            position_size = {}
            for cur, weight in cleaned_weights.items():
                position_size[cur] = weight * portfolio_size

            print("\nPortfolio:\n")
            for i, j in cleaned_weights.items():
                print(f"{i}: {round(position_size[i], 2)}$ ({round(j*100, 2)}%)")
            print(f"\nSum of investments: {int(round(sum(position_size.values()),2))}$")


            plt.bar(list(cleaned_weights.keys()), np.round(np.array(list(cleaned_weights.values()))*100, 2))
            plt.xlabel('Currency')
            plt.ylabel('Percentage')
            plt.title('Percentage currency basket')
            plt.show()

    basket.plot()
    plt.ylabel('Currency to U.S. Dollar')
    plt.show()

window.close()
